In [2]:
import cv2
import numpy as np
from object_detection import ObjectDetection
from deep_sort.deep_sort import Deep
import torch


In [3]:
def detect(model, frame):
  '''Given results from yolov5, extract classs ids, scores and boxes as numpy array'''

  results = model(frame)
  result_pandas = results.pandas().xyxy[0]
  print(result_pandas)
  result_list = results.xyxy[0].cpu().detach().tolist()
  class_ids = []
  scores = []
  boxes = []

  for i in range(len(result_list)):
    class_ids.append(result_list[i][5])
    scores.append(result_list[i][4])
    boxes.append(result_list[i][0:4])

  class_ids = np.asarray(class_ids, dtype=np.int32)
  scores = np.asarray(scores, dtype=np.float64)
  boxes = np.asarray(boxes, dtype=np.float64)

  return class_ids, scores, boxes

In [4]:
# Load Object Detection
yolov4 = ObjectDetection("dnn_model/yolov4.weights", "dnn_model/yolov4.cfg")
yolov4.load_class_names("dnn_model/classes.txt")
yolov4.load_detection_model(image_size=832, # 416 - 1280
                        nmsThreshold=0.4,
                        confThreshold=0.3)
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s')


# Load Object Tracking Deep Sort
deep = Deep(max_distance=0.7,
            nms_max_overlap=1,
            n_init=3,
            max_age=15,
            max_iou_distance=0.7)
tracker = deep.sort_tracker()

Loading Object Detection with params:
image_size=(832, 832)
nmsThreshold=0.4
confThreshold=0.3


Using cache found in /Users/freebox/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-6-23 Python-3.9.7 torch-1.13.0.dev20220710 MPS

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Metal device set to: Apple M1 Pro


2022-07-18 15:42:13.954967: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-18 15:42:13.955687: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
cap = cv2.VideoCapture("videos/city_traffic.mp4")

In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    """ 1. Object Detection per frame"""
    # Question : Insert new model but will give error because its running on mps (need nvidia gpu)
    # >>>>>>>>>>>>>>>>>>>>>>>>>> yoloV5 >>>>>>>>>>>>>>>>>>
    # (class_ids, scores, boxes) = detect(yolov5, frame)
    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    
    (class_ids, scores, boxes) = yolov4.detect(frame)
    print("class_ids>>>>>>>>>>>>>>", class_ids)
    print(type(class_ids))
    print(class_ids.shape)
    print("scores>>>>>>>>>>>>>>", scores)
    print(scores.shape)
    print("boxes>>>>>>>>>>>>>>", boxes)
    print(boxes.shape)
    
    """ 2. Object Tracking """
    features = deep.encoder(frame, boxes)
    detections = deep.Detection(boxes, scores, class_ids, features)

    tracker.predict()
    (class_ids, object_ids, boxes) = tracker.update(detections)

    for class_id, object_id, box in zip(class_ids, object_ids, boxes):

        (x, y, x2, y2) = box
        class_name = yolov4.classes[class_id]
        color = yolov4.colors[class_id]


        cv2.rectangle(frame, (x, y), (x2, y2), color, 2)
        cv2.rectangle(frame, (x, y), (x + len(class_name) * 20, y - 30), color, -1)
        cv2.putText(frame, class_name + " " + str(object_id), (x, y - 10), 0, 0.75, (255, 255, 255), 2)

    # cv2.imshow("Frame", frame)

    # key = cv2.waitKey(1)
    # if key == 27:
    #     break
    break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@165.862] global /Users/runner/miniforge3/conda-bld/libopencv_1644893998420/work/modules/dnn/src/dnn.cpp (1483) setUpNet DNN module was not built with CUDA backend; switching to CPU



systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

class_ids>>>>>>>>>>>>>> [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 9 9 9 9 9 9 9 9]
<class 'numpy.ndarray'>
(38,)
scores>>>>>>>>>>>>>> [    0.90133     0.77655     0.71734     0.70119     0.69992     0.67268     0.65925     0.65782     0.64159     0.62191     0.57465     0.48482     0.40241     0.37371     0.33027     0.99181     0.98656     0.98567     0.97104      0.9508     0.94437     0.93179     0.84599     0.82665     0.61489     0.60572
     0.59809     0.57404     0.51404      0.4943     0.73316     0.61462     0.52944     0.49558     0.47663     0.43839     0.36959     0.32532]
(38,)
boxes>>>>>>>>>>>>>> [[ 764  422    8   16]
 [ 315  560   16   21]
 [ 230  568   11   23]
 [ 543  587   10   20]
 [ 286  474   11   19]
 [ 569  572   11   24]
 [ 785  420    8   20]
 [ 349  569   12   23]
 [ 434  587   11   22]
 [ 744  427    8   16]
 [ 677  626   12   25]
 [ 494  570   11   19]
 [ 661  629   10   22]
 [ 291  572   11

2022-07-18 15:44:19.051800: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-18 15:44:19.052865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
